In [1]:
import requests
obamasample = requests.get("https://obamawhitehouse.archives.gov/the-press-office/remarks-president-welcoming-senior-staff-and-cabinet-secretaries-white-house")

In [3]:
obamasample.text

'<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->\n<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->\n<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->\n<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->\n<!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html  lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video#"><!--<![endif]-->\n\n<head>\n  <!--[if IE]><![endif]-->\n<!--[if IE]><meta http-equiv="X-UA-Compatibl

In [4]:
from bs4 import BeautifulSoup

In [5]:
osoup = BeautifulSoup(obamasample.text)

/Users/pauliglot/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [6]:
odate = osoup.find("meta",  property="article:published_time")

In [8]:
odate

<meta content="2009-01-21T00:00:00-05:00" property="article:published_time"/>

In [9]:
odate["content"]

'2009-01-21T00:00:00-05:00'

In [12]:
from dateutil import parser
od=parser.parse(odate["content"])

In [13]:
od

datetime.datetime(2009, 1, 21, 0, 0, tzinfo=tzoffset(None, -18000))

In [14]:
trumpsample=requests.get("https://www.whitehouse.gov/the-press-office/2017/07/21/remarks-president-trump-visit-survivors-uss-arizona")

In [20]:
tsoup = BeautifulSoup(trumpsample.text)
td = parser.parse(tsoup.find("meta",  property="article:published_time")["content"])

/Users/pauliglot/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [21]:
td

datetime.datetime(2017, 7, 21, 16, 51, 11, tzinfo=tzoffset(None, -14400))

In [22]:
td > od

True

In [24]:
od > td

False

Ok, so I can compare dates.  Can I construct my own date?

In [25]:
from datetime import datetime
datetime.strptime("2009-07-21", "%Y-%m-%d")

datetime.datetime(2009, 7, 21, 0, 0)

So strategy: scrape all the speeches from both obama and trump and dump in a dataset, then compare by date.

In [30]:
otexts = osoup.find_all("div", attrs={"class": "field-type-text-long"})    

In [31]:
len(otexts)

1

In [32]:
otexts[0].text

'\n\nTHE WHITE HOUSE   Office of the Press Secretary  __________________________________________________________________________________________________  For Immediate Release January 21, 2009  REMARKS BY THE PRESIDENT IN WELCOMING SENIOR STAFF AND CABINET SECRETARIES TO THE WHITE HOUSE    Room 450  Eisenhower Executive Office Building   1:18 P.M. EST   THE PRESIDENT: Hello, everybody. Please be seated. Still getting used to that whole thing. (Laughter.) Please be seated. Thank you so much. I wanted to get everyone together on the first day to welcome you to the White House.    From our vantage point yesterday you couldn\'t help but be inspired by the sight of Americans as far as the eye could see. They were there because they believe this is a moment of great change in America, a time for reinvigorating our democracy and remaking our country. They\'ve entrusted all of us with a great responsibility. And so today I\'d like to talk with you about our responsibility to keep that trust.  

In [33]:
ttexts = tsoup.find_all("div", attrs={"class": "field-type-text-long"})  

In [34]:
len(ttexts)

1

In [36]:
ttexts[0].text

"\n\n\n\tOval Office\n\n\t2:51 P.M. EDT\n\n\n\tTHE PRESIDENT: \xa0Thank you very much. \xa0Today it's my distinct privilege to welcome to the White House three of the five living survivors of the USS Arizona. \xa0This is their first time to our nation's capital -- Ken, Lauren, and Don. \xa0I hope this trip does honor to you and your truly heroic service and we wanted to thank you all. \xa0That is so good. \xa0Thank you very much. \xa0Thank you for giving me the nicest hardware, plus a beautiful patch. \xa0Thank you.\n\n\tFor these three World War II veterans, December 7th, 1941, the brutal attack on Pearl Harbor is forever seared into their memories. \xa0It's also seared into America's memory because, on that grim day, this mighty nation was roused to defend freedom itself.\n\n\tEach of them has a harrowing story of courage to share. \xa0They tell us of the American spirit under fire, and of the will of our people to defeat threats to our nation and to the civilized world. \xa0\n\n\tOn

In [37]:
print(osoup.find("title").text)
print(tsoup.find("title").text)

Remarks of the President in Welcoming Senior Staff and Cabinet Secretaries to the White House | whitehouse.gov
Remarks by President Trump at Visit with Survivors from the USS Arizona | whitehouse.gov


In [46]:
# ok this gives me a parser for a url

def probsprez(speechtitle):
    st = speechtitle.lower()
    cues = ["by the president",
           "by president",
           "weekly address",
           "of the president",
           "of president",
           "president barack",
           "president donald",
           "president obama",
           "president trump"]
    return any(x in st for x in cues)

def speech_url_to_dict(url, president):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    date = parser.parse(soup.find("meta",  property="article:published_time")["content"])
    title = soup.find("title").text
    content = soup.find("div", attrs={"class": "field-type-text-long"}).text
    probpre = probsprez(title)
    return {"url": url, "president": president, "actual_president_guess": probpre, "date": date, "title": title, "content": content}

In [47]:
print(speech_url_to_dict("https://obamawhitehouse.archives.gov/the-press-office/remarks-president-after-meeting-with-house-republican-caucus-12709", "Obama"))

{'date': datetime.datetime(2009, 1, 27, 0, 0, tzinfo=tzoffset(None, -18000)), 'president': 'Obama', 'url': 'https://obamawhitehouse.archives.gov/the-press-office/remarks-president-after-meeting-with-house-republican-caucus-12709', 'title': 'Remarks by the President After Meeting With House Republican Caucus, 1/27/09 | whitehouse.gov', 'actual_president_guess': True, 'content': "\n\nTHE WHITE HOUSE   Office of the Press Secretary  _________________________________________________________________  For Immediate Release January 27, 2009  REMARKS BY THE PRESIDENT AFTER MEETING WITH HOUSE REPUBLICAN CAUCUS   Ohio Clock Corridor  U.S. Capitol   1:41 P.M. EST   THE PRESIDENT: Hello, everybody. We had a very constructive meeting with the House members, members of the Republican Caucus. I'm a little bit late for my Senate colleagues -- former Senate colleagues.   And the main message I have is that the statistics every day underscore the urgency of the economic situation. The American people ex

In [48]:
print(speech_url_to_dict("https://www.whitehouse.gov/the-press-office/2017/07/17/remarks-vice-president-christians-united-israel-washington-summit", "Trump"))

{'date': datetime.datetime(2017, 7, 17, 22, 4, tzinfo=tzoffset(None, -14400)), 'president': 'Trump', 'url': 'https://www.whitehouse.gov/the-press-office/2017/07/17/remarks-vice-president-christians-united-israel-washington-summit', 'title': 'Remarks by the Vice President at Christians United for Israel Washington Summit | whitehouse.gov', 'actual_president_guess': False, 'content': "\n\n\n\tWashington Convention Center\n\tWashington, D.C.\xa0\n\n\t8:48 P.M. EDT\n\n\tTHE VICE PRESIDENT: \xa0Hello, CUFI! \xa0(Applause.) \xa0Thank you, Pastor John Hagee, Ambassador Dermer, distinguished members of Congress, honored guests, my fellow believers and friends, it is humbling for me to be before you today to join you at the 12th annual summit of the largest pro-Israel organization in the United States of America -- Christians United for Israel! \xa0(Applause.)\xa0\n\n\tIt’s great to be back with so many friends. \xa0And to all of you -- I bring greetings from another friend. \xa0I just left him

ok, will need error handling for missing stuff, but otherwise good.

In [49]:
# onto the data pages
def extract_links(soup):
    ldivs = soup.find_all("div", attrs={"class": "views-field-title"})
    links = [div.find("a")["href"] for div in ldivs]

In [51]:
print(extract_links(BeautifulSoup(requests.get("https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=472").text)))

None


/Users/pauliglot/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [52]:
# hmm.
opage = requests.get("https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=472").text

In [53]:
opage

'<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->\n<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->\n<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->\n<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->\n<!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html  lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video#"><!--<![endif]-->\n\n<head>\n  <!--[if IE]><![endif]-->\n<!--[if IE]><meta http-equiv="X-UA-Compatibl

In [54]:
osoup = BeautifulSoup(opage)

/Users/pauliglot/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [55]:
osoup

<!DOCTYPE html>

<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->
<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->
<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->
<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->
<!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video#"><!--<![endif]-->
<head>
<!--[if IE]><![endif]-->
<!--[if IE]><meta http-equiv="X-UA-Compatible" content="I

In [57]:
odivs = osoup.find_all("div", attrs={"class": "views-field-title"})

In [58]:
[div.find("a")["href"] for div in odivs]

['/the-press-office/remarks-first-lady-and-mrs-lilly-ledbetter-reception-after-bill-signing',
 '/the-press-office/remarks-president-after-meeting-with-secretary-defense-and-joint-chiefs-staff-12809',
 '/the-press-office/remarks-president-barack-obama-economy',
 '/the-press-office/remarks-president-after-meeting-with-house-republican-caucus-12709',
 '/the-press-office/weekly-address-president-barack-obama-discusses-new-white-house-report-american-rec',
 '/the-press-office/remarks-president-bipartisan-meeting-economy',
 '/the-press-office/president-barack-obamas-inaugural-address',
 '/the-press-office/remarks-president-welcoming-senior-staff-and-cabinet-secretaries-white-house']

In [59]:
extract_links(osoup)

In [61]:
# oh yeah, I'm in a language that doesn't return the last statement. rofl.
def extract_links(soup):
    ldivs = soup.find_all("div", attrs={"class": "views-field-title"})
    links = [div.find("a")["href"] for div in ldivs]
    return links
extract_links(osoup)

['/the-press-office/remarks-first-lady-and-mrs-lilly-ledbetter-reception-after-bill-signing',
 '/the-press-office/remarks-president-after-meeting-with-secretary-defense-and-joint-chiefs-staff-12809',
 '/the-press-office/remarks-president-barack-obama-economy',
 '/the-press-office/remarks-president-after-meeting-with-house-republican-caucus-12709',
 '/the-press-office/weekly-address-president-barack-obama-discusses-new-white-house-report-american-rec',
 '/the-press-office/remarks-president-bipartisan-meeting-economy',
 '/the-press-office/president-barack-obamas-inaugural-address',
 '/the-press-office/remarks-president-welcoming-senior-staff-and-cabinet-secretaries-white-house']

In [64]:
def extract_pages(soup):
    pgs = soup.find("li", attrs={"class": "pager-current"}).text.split(" ")
    return {"current": pgs[0], "total": pgs[2]}
extract_pages(osoup)

{'current': '473', 'total': '473'}

In [65]:
tsoup = BeautifulSoup(requests.get("https://www.whitehouse.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=0").text)

/Users/pauliglot/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [66]:
extract_pages(tsoup)

{'current': '1', 'total': '32'}

In [67]:
extract_links(tsoup)

['/the-press-office/2017/08/08/remarks-president-trump-briefing-opioid-crisis',
 '/the-press-office/2017/08/04/remarks-vice-president-young-americas-foundation',
 '/the-press-office/2017/08/04/president-donald-j-trumps-weekly-address',
 '/the-press-office/2017/08/04/remarks-president-trump-fema-headquarters',
 '/the-press-office/2017/08/03/remarks-president-trump-department-veterans-affairs-telehealth-event',
 '/the-press-office/2017/08/02/remarks-president-trump-senator-tom-cotton-and-senator-david-perdue',
 '/the-press-office/2017/08/02/remarks-vice-president-adriatic-charter-summit',
 '/the-press-office/2017/08/01/remarks-vice-president-noble-partner-participants',
 '/the-press-office/2017/08/01/remarks-vice-president-and-georgian-prime-minister-joint-press',
 '/the-press-office/2017/07/31/remarks-president-trump-presentation-medal-honor-private-first-class']

Sweet, I'm good, should be relatively easy to get there from here

In [69]:
starting_pages = {"Obama": "https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=0", "Trump": "https://www.whitehouse.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=0"}

In [77]:
def setup_root_page(starting_page):
    html = requests.get(starting_page).text
    soup = BeautifulSoup(html, "html.parser")
    root = "".join(starting_page.partition(".gov")[:-1])
    this_page_links = [root + link for link in extract_links(soup)]
    numpages = int(extract_pages(soup)["total"])
    newpages = [starting_page[:-1] + str(x) for x in range(1, numpages)]
    return {"initial_links": this_page_links, "pages_to_fetch": newpages, "urlroot": root}
    

In [78]:
obamaroot = setup_root_page(starting_pages["Obama"])

In [79]:
obamaroot

{'initial_links': ['https://obamawhitehouse.archives.gov/the-press-office/2017/01/18/remarks-president-final-press-conference',
  'https://obamawhitehouse.archives.gov/the-press-office/2017/01/18/remarks-vice-president-joe-biden-world-economic-forum',
  'https://obamawhitehouse.archives.gov/the-press-office/2017/01/17/remarks-vice-president-joe-biden-cancer-moonshot',
  'https://obamawhitehouse.archives.gov/the-press-office/2017/01/17/remarks-vice-president-joe-biden-ukrainian-president-petro-poroshenko',
  'https://obamawhitehouse.archives.gov/the-press-office/2017/01/16/remarks-president-honoring-world-series-champion-chicago-cubs',
  'https://obamawhitehouse.archives.gov/the-press-office/2017/01/14/weekly-address-honor-serving-you-president',
  'https://obamawhitehouse.archives.gov/the-press-office/2017/01/12/remarks-president-and-vice-president-presentation-medal-freedom-vice',
  'https://obamawhitehouse.archives.gov/the-press-office/2017/01/12/remarks-vice-president-nuclear-securi

In [80]:
trumproot = setup_root_page(starting_pages["Trump"])

In [81]:
trumproot

{'initial_links': ['https://www.whitehouse.gov/the-press-office/2017/08/08/remarks-president-trump-briefing-opioid-crisis',
  'https://www.whitehouse.gov/the-press-office/2017/08/04/remarks-vice-president-young-americas-foundation',
  'https://www.whitehouse.gov/the-press-office/2017/08/04/president-donald-j-trumps-weekly-address',
  'https://www.whitehouse.gov/the-press-office/2017/08/04/remarks-president-trump-fema-headquarters',
  'https://www.whitehouse.gov/the-press-office/2017/08/03/remarks-president-trump-department-veterans-affairs-telehealth-event',
  'https://www.whitehouse.gov/the-press-office/2017/08/02/remarks-president-trump-senator-tom-cotton-and-senator-david-perdue',
  'https://www.whitehouse.gov/the-press-office/2017/08/02/remarks-vice-president-adriatic-charter-summit',
  'https://www.whitehouse.gov/the-press-office/2017/08/01/remarks-vice-president-noble-partner-participants',
  'https://www.whitehouse.gov/the-press-office/2017/08/01/remarks-vice-president-and-georg

In [86]:
def links_to_parse(prezroot):
    initlinks = prezroot["initial_links"][:]
    pages_to_fetch = prezroot["pages_to_fetch"][:]
    root = prezroot["urlroot"]
    for page in pages_to_fetch:
        html = requests.get(page).text
        soup = BeautifulSoup(html, "html.parser")
        curlinks = [root + link for link in extract_links(soup)]
        initlinks = initlinks + curlinks
    return initlinks
    

In [87]:
trumplinks = links_to_parse(trumproot)

In [88]:
len(trumplinks)

318

In [89]:
trumplinks

['https://www.whitehouse.gov/the-press-office/2017/08/08/remarks-president-trump-briefing-opioid-crisis',
 'https://www.whitehouse.gov/the-press-office/2017/08/04/remarks-vice-president-young-americas-foundation',
 'https://www.whitehouse.gov/the-press-office/2017/08/04/president-donald-j-trumps-weekly-address',
 'https://www.whitehouse.gov/the-press-office/2017/08/04/remarks-president-trump-fema-headquarters',
 'https://www.whitehouse.gov/the-press-office/2017/08/03/remarks-president-trump-department-veterans-affairs-telehealth-event',
 'https://www.whitehouse.gov/the-press-office/2017/08/02/remarks-president-trump-senator-tom-cotton-and-senator-david-perdue',
 'https://www.whitehouse.gov/the-press-office/2017/08/02/remarks-vice-president-adriatic-charter-summit',
 'https://www.whitehouse.gov/the-press-office/2017/08/01/remarks-vice-president-noble-partner-participants',
 'https://www.whitehouse.gov/the-press-office/2017/08/01/remarks-vice-president-and-georgian-prime-minister-joint-p

In [90]:
obamalinks = links_to_parse(obamaroot)

In [91]:
import json
with open("prezlinks.json", "w") as pj:
    json.dump({"Obama": obamalinks, "Trump": trumplinks}, pj)

Ok!  Now all I have to do is grab every link with `speech_url_to_dict(url, president)` and shove it in a dataframe and we have a dataset!

In [92]:
from pandas import DataFrame
speeches = []
for link in trumplinks:
    speeches.append(speech_url_to_dict(link, "Trump"))

for link in obamalinks:
    speeches.append(speech_url_to_dict(link, "Obama"))

df = DataFrame(speeches)
df.to_csv('allspeeches.csv', index=False)

KeyboardInterrupt: 

In [93]:
len(speeches)

4478

In [94]:
df = DataFrame(speeches)
df.to_csv('partial-speeches.csv', index=False)

In [95]:
len(obamalinks) + len(trumplinks)

5046

In [96]:
speeches[-1]

{'actual_president_guess': True,
 'content': "\n\n\n\n\nTHE WHITE HOUSE\n\n\t\t\tOffice of the Press Secretary\n\t\t\t___________________________________________________________________________\n\t\t\tFor Immediate Release\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0November 3, 2009\n\t\t\t\xa0\n\n\nREMARKS BY PRESIDENT OBAMA,\n\tPRIME MINISTER FREDRIK REINFELDT OF SWEDEN,\n\tEUROPEAN COMMISSION PRESIDENT JOSÉ MANUEL BARROSO,\n\tAND EUROPEAN COUNCIL HIGH REPRESENTATIVE JAVIER SOLANA\n\tAFTER MEETING\nCabinet Room\n\n\t4:11 P.M. EST\nPRESIDENT OBAMA:\xa0 Well, I want to welcome, for the second day in a row, Prime Minister Reinfeldt.\xa0 I also want to recogn

In [97]:
got_urls = set([x["url"] for x in speeches])

In [98]:
missing = set(obamalinks) - got_urls

In [99]:
len(missing)

559

In [101]:
# that's interesting. 4478 + 559 = 5037, not 5046  Am I somehow missing some trump ones too?? 
# or is it just from the initial links, there should be 8 or 9 there?
print(set(trumplinks) - got_urls)


set()


In [103]:
with open("missingurls.json", "w") as mj:
    json.dump(list(missing), mj)

In [104]:
# ok, I'm going to have to do some error handling or something to grab the last few...
import time
speeches2 = []
def careful_speech_url_to_dict(url, president):
    time.sleep(0.5)
    try:
        page = requests.get(url, timeout=2)
    except Exception as e:
        return {"error": str(e), "failed_url": url}
    soup = BeautifulSoup(page.text, "html.parser")
    date = parser.parse(soup.find("meta",  property="article:published_time")["content"])
    title = soup.find("title").text
    content = soup.find("div", attrs={"class": "field-type-text-long"}).text
    probpre = probsprez(title)
    return {"url": url, "president": president, "actual_president_guess": probpre, "date": date, "title": title, "content": content}

In [105]:
for link in missing:
    speeches2.append(careful_speech_url_to_dict(link, "Obama"))

In [107]:
for item in speeches2:
    if "error" in item:
        print(item["error"])

HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read timeout=2)
HTTPSConnectionPool(host='obamawhitehouse.archives.gov', port=443): Read timed out. (read t

In [113]:
df2 = DataFrame(speeches2)
df2.to_csv('partial-speeches2.csv', index=False)

In [109]:
stillmissing = [x["failed_url"] for x in speeches2 if "error" in x]

In [110]:
stillmissing

['https://obamawhitehouse.archives.gov/the-press-office/remarks-president-town-hall-health-care-grand-junction-colorado',
 'https://obamawhitehouse.archives.gov/the-press-office/weekly-address-president-obama-celebrates-independence-day-and-american-spirit',
 'https://obamawhitehouse.archives.gov/the-press-office/weekly-address-president-obama-says-recovery-act-creating-jobs-and-strengthening-ec',
 'https://obamawhitehouse.archives.gov/the-press-office/weekly-address-president-obama-calls-health-insurance-reform-key-stronger-economy-a',
 'https://obamawhitehouse.archives.gov/the-press-office/weekly-address-president-obama-calls-hails-progress-health-insurance-reform-despite',
 'https://obamawhitehouse.archives.gov/the-press-office/remarks-president-keep-indiana-blue-fundraiser-indianapolis-5172009',
 'https://obamawhitehouse.archives.gov/the-press-office/remarks-presidentat-dnc-fundraising-reception',
 'https://obamawhitehouse.archives.gov/the-press-office/remarks-president-obama-and-n

In [111]:
len(stillmissing)

18

In [112]:
# ok, fine, that's better.
with open("still-missing-round-2.json", "w") as sm:
    json.dump(stillmissing, sm)

In [114]:
# let's see if it'll let me try again a few minutes later.
try3 = []
for link in stillmissing:
    try3.append(careful_speech_url_to_dict(link, "Obama"))

In [115]:
try3

[{'actual_president_guess': True,
  'content': '\n\nTHE WHITE HOUSE Office of the Press Secretary  _______________________________________________________________________________  For Immediate Release\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 August 15, 2009 REMARKS BY THE PRESIDENT  IN TOWN HALL ON HEALTH CARE Central High school  Grand Junction, Colorado 3:44 P.M. MDT THE PRESIDENT:\xa0 Thank you.\xa0 Thank you.\xa0 Hello, Grand Junction!\xa0 (Applause.)\xa0 Thank you so much.\xa0 Thank you.\xa0 Everybody please have a seat.\xa0 It is good to be back in Southwest Colorado.\xa0 (Applause.)\xa0 Last time I was here I had some really good peaches.\xa0 (Laughter.)\xa0 Somehow, though, Michelle and the girls got to go pick peaches -- and I\'m hoping they bring some back for me. It is nice to take

In [116]:
for item in try3:
    if "error" in item:
        print(item["error"])

In [117]:
df3 = DataFrame(try3)
df3.to_csv('partial-speeches3.csv', index=False)